In [78]:
from ortools.linear_solver import pywraplp
import pandas as pd

In [79]:
# MAKE SURE THAT THE FORM HAS BEEN EMPTIED FIRST.
# MAKE SURE THAT THE HEADER HAVE BEEN CLEANED-UP.
raw_data = pd.read_csv('/Users/sahuguet/Downloads/NYUAD-demo.csv', header=1)

raw_data[0:5]

,ResponseID,ResponseSet,StartDate,EndDate,Finished,nyuad-logo,Please indicate the project(s) you want to work on,Your name,Your gender,Are you an international participant?,...,Please rank the projects (top = most preferred)-ClimateViz,Please rank the projects (top = most preferred)-Celebreat,Please rank the projects (top = most preferred)-Fahmt,Please rank the projects (top = most preferred)-Tindercise,Please rank the projects (top = most preferred)-Tasaka,Please rank the projects (top = most preferred)-Sadiki,Please rank the projects (top = most preferred)-Beacon Manara,Please rank the projects (top = most preferred)-VR Refugees,Please rank the projects (top = most preferred)-Protector,Unnamed: 28
0,R_30x9Uwhs6nzg7jR,Default Response Set,2016-04-15 08:18:57,2016-04-15 08:19:31,1,1,1,Quan Vuong,1,2,...,11,10,1,6,13,8,4,7,14,NaN
1,R_x3LrSGQNyflcl7X,Default Response Set,2016-04-15 08:18:50,2016-04-15 08:20:26,1,1,1,Qandeel Tariq,2,1,...,6,3,1,2,11,10,5,15,8,NaN
2,R_QgYh7SqqGS4E9mp,Default Response Set,2016-04-15 08:18:49,2016-04-15 08:24:04,1,1,1,Lingliang Zhang,1,2,...,4,5,1,13,6,8,15,12,9,NaN
3,R_AiKdCU7j1jlJrzz,Default Response Set,2016-04-15 08:22:44,2016-04-15 08:24:23,1,1,1,Hazem Ibrahim,1,2,...,3,10,15,8,14,12,11,7,13,NaN
4,R_RUiyKZ2TcqNuFON,Default Response Set,2016-04-15 08:20:53,2016-04-15 08:24:55,1,1,1,Matthew Volk,1,1,...,6,11,1,15,12,13,5,8,4,NaN


In [80]:
STUDENTS = raw_data['Your name']
data = raw_data.set_index('Your name')
GENDER = data['Your gender'] -1
INTERNATIONAL = data['Are you an international participant?'] -1
NOT_UAE = (data['Are you from the UAE?'] -1) / 2
NOT_NYU_AD = (data['Are you from NYU AD?'] -1) / 2
NO_SKILLS = data['Do you consider yourself an experienced developer?'] -1

PROJECTS = raw_data.columns[13:28] # We limit the number of projects.

SCORING = { 1:1, 2:100, 3:150}
SCORING.update({ i:156+i for i in range(4,25)})



In [81]:
NOT_UAE.sum()

49.0

In [82]:
#STUDENTS = pd.concat([STUDENTS[0:72], STUDENTS[76:76]])
STUDENTS = STUDENTS

# Average number of students.
RATIO = int(len(STUDENTS)/len(PROJECTS)) + 1
print ("Avg team size: %d " % RATIO)
NO_SKILLS
print (len(STUDENTS))

Avg team size: 6 
87


In [83]:
solver = pywraplp.Solver('StudentProjectGridCBC', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
#solver = pywraplp.Solver('StudentProjectGridCBC', pywraplp.Solver.GLPK_MIXED_INTEGER_PROGRAMMING)

matches = {}
for student in STUDENTS:
    for project in PROJECTS:
        matches[student, project] = solver.IntVar(0, 1, 'matches[%s,%s]' % (student, project))
        
z = solver.Sum([SCORING[data[project][student]] *  matches[student, project]
                 for student in STUDENTS
                 for project in PROJECTS])


######################
#### Constraints #####
######################

# One person per project
for student in STUDENTS:
    solver.Add(solver.Sum([matches[student, project] for project in PROJECTS]) == 1)

# At least one international student per project
for project in PROJECTS:
    solver.Add(solver.Sum([matches[student, project] * (1-INTERNATIONAL[student]) for student in STUDENTS]) >= 1)

# No more than xx NYU AD students
for project in PROJECTS:
    solver.Add(solver.Sum([matches[student, project] * (1 - NOT_NYU_AD[student]) for student in STUDENTS]) <= 4)

# No more than xx UAE students
for project in PROJECTS:
    solver.Add(solver.Sum([matches[student, project] * (1 - NOT_UAE[student]) for student in STUDENTS]) <= 3)

# Good gender diversity
for project in PROJECTS:
    solver.Add(solver.Sum([matches[student, project] * GENDER[student] for student in STUDENTS]) >= 2)

# Enough skills in the team
for project in PROJECTS:
    solver.Add(solver.Sum([matches[student, project] * (1-NO_SKILLS[student]) for student in STUDENTS]) >= 1)

# Good size balance: more than 2 and less than RATIO.
for project in PROJECTS:
    solver.Add(solver.Sum([matches[student, project] for student in STUDENTS]) >= 3)
    solver.Add(solver.Sum([matches[student, project] for student in STUDENTS]) <= RATIO)
    
#solver.Add(solver.Sum([matches[student, 'Cityzoom'] for student in STUDENTS]) == 7)
    
# Hardcoded assignments for students.
# hard_coded_assignments = {"Tareq si salem": "Crowd Manager", "Rawan ELJAMAL": "Crowd Manager"}
for student in STUDENTS:
    continue
    if student in hard_coded_assignments:
        solver.Add(matches[student, hard_coded_assignments[student]] == 1)

In [84]:
objective = solver.Minimize(z)
solver.Solve()
s = solver.Objective().Value()
s

3244.0

In [85]:
BUCKETS = {}
for project in PROJECTS:
    BUCKETS[project] = []

ASSIGNMENT = 0
for student in STUDENTS:
    for project in PROJECTS:
        if int(matches[student, project].SolutionValue()):
            student_name = student
            if INTERNATIONAL[student] == 0:
                student_name += '-*'
            if GENDER[student] == 1:
                student_name += '-F'
            if NOT_NYU_AD[student] == 0:
                student_name += '-nyu'
            if NOT_UAE[student] == 0:
                student_name += '-uae'
            if NO_SKILLS[student] == 0:
                student_name += '-xp'
            student_name += " (%d)" % data[project][student]
            BUCKETS[project].append(student_name)
            ASSIGNMENT += 1

for bucket in BUCKETS:
    people = BUCKETS[bucket]
    print ("%s (%d)" % (bucket, len(people)))
    print (BUCKETS[bucket])
    print
    
    
print (len(BUCKETS), 'projects.')

Please rank the projects (top = most preferred)-Arabic reCaptcha (6)
['Khalid AlAwar-uae-xp (1)', 'Samr Samir Ali-F-uae-xp (1)', 'Ali Senhaji (1)', 'Rana Abul-Haija-*-F-xp (1)', 'Katia-F (1)', 'Muhammed Al Sayadi-uae-xp (1)']
Please rank the projects (top = most preferred)-Crowd Manager (6)
['Rawan ELJAMAL-F (1)', 'Prasant-*-nyu-uae (1)', 'wasi-uae-xp (1)', 'Hanan Hadad-F-uae (1)', 'Rawane eljamal -F (1)', 'Jane Choe-F (1)']
Please rank the projects (top = most preferred)-Smart Fork (6)
['Amira Gamal-F-xp (1)', 'ImadEddine Toubal-xp (1)', 'Tareq si salem-xp (1)', 'CHORFI IHEB-*-xp (1)', 'Shouq Darwish-F-uae-xp (1)', 'Leanne-F-uae (1)']
Please rank the projects (top = most preferred)-Maktabi (6)
['Ali Aldarwish-uae-xp (1)', 'Kyle Dhillon-* (1)', 'Sara Al Kendi-F-uae (1)', 'Alaa Daffalla-F (1)', 'Nadine Handal-F (1)', 'Jomana jamal obaida-*-F (1)']
Please rank the projects (top = most preferred)-DiploMap (6)
['Paula Dozsa-F-nyu-uae (1)', 'William-nyu-uae (3)', 'Wafa Waheeda Syed-*-F-xp (